In [1]:
import tensorflow as tf
import numpy as np 
from tensorflow import keras


2025-04-19 20:02:37.757142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745085757.770677   25988 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745085757.774597   25988 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-19 20:02:37.790180: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(train_images,train_labels),(test_images,test_labels) = mnist.load_data()

In [4]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)
encoded_train_label = keras.utils.to_categorical(train_labels, num_classes=10)
encoded_test_label = keras.utils.to_categorical(test_labels, num_classes=10)
print(encoded_train_label.shape)
print(encoded_test_label.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
(60000, 784)
(60000,)
(10000, 784)
(10000,)
(60000, 10)
(10000, 10)


# MLP realted stuff

In [5]:
def model_create(input_dim , output_dim , hidden): # hidden here is a array each number represnet the numpber of nurons in each hidden layer
    model = keras.Sequential()
    model.add(keras.layers.Dense(hidden[0], input_shape=(input_dim,),activation='relu')) # input layers with dim 28*28 for our problem

    for nueron in hidden[1:]:
        model.add(keras.layers.Dense(nueron, activation='relu'))

    model.add(keras.layers.Dense(output_dim, activation='softmax'))

    return model
    
    
    

In [6]:
def extract_weight(model):
    weights = []

    for layer in model.layers:
        layer_weights = layer.get_weights()

        weights.extend([w.flatten() for w in layer_weights])
    return np.concatenate(weights)
    

In [7]:
def return_weights_toModel(model,weight_vector):
    #save shapes
    idx=0

    for i, layer in enumerate(model.layers):  # enumerate to make no iterable items iterable reutrn index i here for example
        layer_weight = layer.get_weights()

        new_weights= [] # new list of weights

        for j,w in enumerate(layer_weight): # j index from enumerate i didnt used it because i dont need it 
            
            n_weights = w.size

            new_w = weight_vector[idx:idx + n_weights].reshape(w.shape)

            new_weights.append(new_w)

            idx += n_weights

        layer.set_weights(new_weights)

    return model  # model with updated layers
    

# fitness function

In [8]:
def fitness_function(x_train, y_train, original_model, weight_vector):

    model_copy = keras.models.clone_model(original_model) # model with new wieght to test with 
    model_copy.build(original_model.input_shape) # without build : erorr weight is not defined in next line when i try to set weight

    return_weights_toModel(model_copy, weight_vector)

    y_pred = model_copy.predict(x_train)

    mse = np.mean((y_pred-y_train)**2)

    return mse
    

In [9]:
# test example 
model = model_create(input_dim=28*28 , output_dim=10 , hidden=[100])

#weight_vecotor_test = extract_weight(model)
#print(weight_vecotor_test)

#mse = fitness_function(train_images,encoded_train_label, model, weight_vecotor_test)
#print("mse is : ",mse)

/home/theodoros/.local/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1745085760.666396   25988 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2169 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


# intialzation function

DONT FORGET HERE WHERE YOU SET SEED!!!!


In [28]:
tf.random.set_seed(24)
np.random.seed(24)
def intialize_population(model, population_size, min_val=0 , max_val=1.0):

    weights = extract_weight(model) # to get the length to create the new at the same size

    num_weights = len(weights)

    population = tf.random.uniform(
        shape = (population_size, num_weights),
        minval=min_val,
        maxval=max_val,
        dtype = tf.float32
        
    )

    population_np = population.numpy()


    return population_np
    

In [11]:
# test
NP=50
population=intialize_population(model,NP)

In [12]:
population.shape


(50, 79510)

In [13]:
population

array([[0.10927987, 0.26020873, 0.7633631 , ..., 0.4038297 , 0.8379352 ,
        0.82291996],
       [0.35637295, 0.01772416, 0.4506879 , ..., 0.3724171 , 0.58914685,
        0.8366225 ],
       [0.5912293 , 0.13186228, 0.8393806 , ..., 0.71497154, 0.8859141 ,
        0.4612081 ],
       ...,
       [0.14626813, 0.11659384, 0.6644871 , ..., 0.32443225, 0.02631509,
        0.12299669],
       [0.34714139, 0.3690194 , 0.62691045, ..., 0.93203974, 0.35366738,
        0.23383474],
       [0.25556505, 0.6664616 , 0.03995574, ..., 0.08379412, 0.45954823,
        0.3748058 ]], dtype=float32)

In [14]:
D = len(extract_weight(model))
print("number of genes is : ",D)

number of genes is :  79510


In [15]:
# other parmeters

F = 0.9
CR = 0.5
GEN= 100

# constrains

L= 0
H= 1


In [37]:
# selection for the 3 random parent i prreffered to make it a function for editing later 
def random_vectors(population, current_index):
    
    population_size = population.shape[0]

    # remove the target vectory  

    population2 = list(range(population_size))
    population2.remove(current_index)

    # choosing indices 
    indexs = np.random.choice(population2, size=3 , replace=False)

    v1 = population[indexs[0]]
    v2 = population[indexs[1]]
    v3 = population[indexs[2]]

    return v1, v2, v3



# test 

x1,x2,x3 = random_vectors(population,5)
print(x2)
print(x2.shape)

# dont forget to check glitch with seed here later 
    

[0.60866606 0.10641992 0.0028317  ... 0.9428768  0.8928089  0.42391562]
(79510,)


# mutation and crossover function

In [43]:
def mutate (x1,x2,x3,F):
    v = x3+F*(x1-x2)

    return v


# test 
v = mutate(x1,x2,x3 ,.9)
print(v.shape)
print (v)

(79510,)
[1.0587404  0.9157058  0.75013244 ... 0.12005645 0.26460293 0.56535935]


In [26]:
def evolve(population,GEN,NP,D):
    
    for g in range(GEN):
    
        for j in range(NP):
            # now choosing 3 ranom vectory to calculate 
            v1,v2,v3 = random_vectors(population,j)
        


SyntaxError: incomplete input (2750638937.py, line 7)

In [ ]:
print(x1.shape)